# Normalization pipeline

Die Quelldaten sind aus meinem ersten RAG-Projekt und wurden mit fruendlicher Genehmigung von einer Webseite gescrappt.

**Was zu tun ist:**
- Daten von Marketingaussagen und Firmengeschichte befreien
- Die Specs normalisieren

Das meiste werde ich mit einem LLM machen. Die Ausgangsdaten werden stichprobenartig kontrolliert. Ziel ist ein funktionierendes und korrektes retrival, welches nicht zwingend mit der realen Welt (korrekte technische Daten z.B.) übereinstimmen muss. In einem Realworldprojekt ist das selbstverständlich etwas anderes.

Wärend der Bereinigung werden die Daten in Specs und Descriptions aufgeteilt.

In [1]:
import os
import json
import time
import pandas as pd
from tqdm import tqdm
from mistralai import Mistral
from dotenv import load_dotenv

load_dotenv()

df = pd.read_json('./products_raw.json')

api_key = os.getenv('MISTRAL_API_KEY')
model = 'mistral-medium-2508'
client = Mistral(api_key=api_key)

descs_promt = open('./descs_agent.md', 'r').read()
specs_promt = open('./specs_agent.md', 'r').read()
summs_promt = open('./summs_agent.md').read()

descs_chunks = []
specs_chunks = []
summs_chunks = []

Die Beschreibungen werden über ein LLM bereinigt. Der Systempromt liegt als md-File vor. 

### Funktionen

- **agent_request**: Allgemeine Retrivalfunktion. Fallbackfunktion (wegen fehlenden Specs integriert)
- **format_spec**: Bereitet Embeddingdocs für Specs auf

In [2]:
def agent_request(promt, content, fallback):

    if content == '[]': content = fallback
    
    response = client.chat.complete(
        model = model,
        messages = [
            {
                'role': 'system',
                'content': promt
            },
            {
                'role': 'user',
                'content': content,
            }
        ],
        response_format = {"type": "json_object"}
    )

    return response

def format_spec(product, group_name, specs_list):
    if not specs_list:
        return ''
    
    if '-' in group_name:
        label, unit = group_name.rsplit('-', 1)
        header = f"{product} {label} ({unit})"
    else:
        header = group_name
    
    spec_parts = []

    for key, value in specs_list.items():
        readable_key = key.replace('_', ' ')
        spec_parts.append(f"{readable_key}: {value}")

    return f"{header} - {', '.join(spec_parts)}"

# Erstellung der Chunks

Die Daten werden an ein LLM geliefert und wir erhalten nur die erzeugen Artefarkte. Diese werden in des Schema der Chunks integriert:

In [3]:
for index, row in tqdm(df.iterrows(), total=len(df)):

    summs_response = agent_request(summs_promt, json.dumps(row.to_dict(), ensure_ascii=False), False)
    descs_response = agent_request(descs_promt, row['description'], False)
    specs_response = agent_request(specs_promt, json.dumps(row['specs'], ensure_ascii=False), row['description'])

    summs_result = json.loads(summs_response.choices[0].message.content)                              
    descs_result = json.loads(descs_response.choices[0].message.content)
    specs_result = json.loads(specs_response.choices[0].message.content)

    # Debug only
    # print(summs_result)
    # print(descs_result)
    # print(specs_result)

    category = summs_result['category']
    manufacturer = summs_result['manufacturer']
    title = summs_result['title']

    product = f"{manufacturer} {title}"

    product_metadata = {
        'product_id': row['id'],
        'product_title': title,
        'product_url': row['url'],
        'product_category': category,
        'product_manufacturer': manufacturer,
        'product_price': row['price'],
        'product_sku': row['order_number']
    }

    summs_chunks.append({
        'document': summs_result['summary'],
        'metadata': {
            'chunk_id': f"{row['id']}_summ_0",
            'chunk_type': 'description',
            **product_metadata
        }
    })

    for i, pararagraph in enumerate(descs_result):
        descs_chunks.append({
            'document': pararagraph,
            'metadata': {
                'chunk_id': f"{row['id']}_desc_{i}",
                'chunk_type': 'description',
                **product_metadata
            }
        })

    for i, (group_name, specs_list) in enumerate(specs_result.items()):
        if specs_list:
            specs_chunks.append({
                'document': format_spec(product, group_name, specs_list),
                'metadata': {
                    'chunk_id': f"{row['id']}_spec_{i}",
                    'chunk_type': 'specs',
                    **product_metadata
                }
            })

    # Testing only
    # if index == 65: break
    # Debug only
    # print(summs_chunks)
    # print(descs_chunks)
    # print(specs_chunks)

summs_df = pd.DataFrame(summs_chunks)
descs_df = pd.DataFrame(descs_chunks)
specs_df = pd.DataFrame(specs_chunks)

  1%|          | 1/151 [00:34<1:25:51, 34.34s/it]

{'summary': 'Der Kirsch LABO-288 PRO-ACTIVE ist ein Laborkühlschrank mit 280 Litern Nutzvolumen und einer Temperaturregelung von ca. 0 bis +15 °C. Der Kühlschrank verfügt über eine statisch belüftete, geräuscharme und energiesparende Kältemaschine, die hermetisch gekapselt ist. Die automatische Abtauung wird durch zeitlich begrenzte und thermische Umkehr des Kältemittelkreislaufs überwacht. Der Innenraum besteht aus glattem Aluminium mit farbloser Schutzbeschichtung und ist mit drei kunststoffbeschichteten Rosten ausgestattet. Der Laborkühlschrank ist umluftgekühlt mit einem Querstromgebläse, das beim Öffnen der Tür automatisch abschaltet, was für eine gleichmäßige Temperierung sorgt und das Temperaturgefälle reduziert. Zusätzlich verfügt der Kühlschrank über ein akustisches und optisches Alarmsignal bei abweichenden Temperaturen sowie eine USB-Schnittstelle und einen potentialfreien Kontakt zum Anschluss an die zentrale Leittechnik.', 'category': 'Laborkühlschrank', 'manufacturer': 'K

  1%|▏         | 2/151 [01:09<1:25:49, 34.56s/it]

{'summary': 'Der Kirsch FROSTER LABEX-530 ULTIMATE ist ein Laborgefrierschrank mit explosionsgeschütztem Innenraum, speziell für die Lagerung von leicht entzündlichen und explosiven Stoffen konzipiert. Mit einem Nutzvolumen von 500 Litern und einer Temperaturregelung von -10°C bis -30°C bietet er präzise Kühlbedingungen. Der Gefrierschrank verfügt über eine PRO-ACTIVE Steuerungselektronik, die eine permanente Überwachung der Leistungsdaten und Alarmierung bei Abweichungen ermöglicht. Zusätzlich ist er mit einer automatischen Abtauung, einer Schnellfrosteinrichtung und einer Umluftkühlung mit Axialgebläse ausgestattet, was eine hohe Temperaturkonstanz bei geringem Energieverbrauch gewährleistet. Der Kirsch FROSTER LABEX-530 ULTIMATE ist ideal für den Einsatz in Laboren, Kliniken und im Gesundheitswesen.', 'category': 'Labortiefkühlschrank', 'manufacturer': 'Kirsch', 'title': 'FROSTER LABEX-530 ULTIMATE'}
['Der Kirsch FROSTER LABEX-530 Laborgefrierschrank ist speziell für die sichere Lag

  2%|▏         | 3/151 [01:47<1:30:06, 36.53s/it]

...

{'summary': 'Der Liebherr SRPvh 8401 Perfection ist ein hochmodernes Laborkühlgerät mit dynamischer Umluftkühlung und einem Bruttovolumen von 871 Litern. Der Temperaturbereich lässt sich zwischen -2°C und +16°C einstellen, wobei die Temperaturabweichung maximal ±2°C beträgt. Das Gerät ist mit einem Alarm-Relais, einer WiFi-/LAN-Schnittstelle und einer integrierten SmartMonitoring-Anbindung ausgestattet, die eine kontinuierliche Überwachung und Dokumentation der Lagertemperatur ermöglicht. Mit einem jährlichen Energieverbrauch von 220 kWh und einer Nutzungsdauer von bis zu 15 Jahren bietet der SRPvh 8401 eine nachhaltige und zuverlässige Lösung für anspruchsvolle Anwendungen in medizinischen Einrichtungen, Forschungslaboren und Apotheken.', 'category': 'Laborkühlschrank', 'manufacturer': 'Liebherr', 'title': 'SRPvh 8401 Perfection'}
['Der Liebherr SRPvh 8401 ist ein Laborkühlschrank, der speziell für medizinische Einrichtungen, Forschungslabore und Apotheken entwickelt wurde. Mit einem 

 99%|█████████▉| 150/151 [1:37:31<00:37, 37.05s/it]

{'summary': 'Der Kirsch FROSTER LABEX-96 PRO-ACTIVE ist ein Laborgefrierschrank, speziell für die Lagerung von leicht entzündlichen und explosiven Stoffen konzipiert. Mit einem explosionsgeschützten Innenraum, der die Sicherheitsanforderungen der Zone 2 erfüllt und ein TÜV-Zertifikat für Zündquellfreiheit besitzt, bietet er höchste Sicherheit. Der Gefrierschrank verfügt über eine geräuscharme, zwangsbelüftete und energiesparende Kältemaschine mit einer Temperaturregelung von -5°C bis -25°C. Weitere Merkmale sind eine Schnellfrosteinrichtung, eine USB-Schnittstelle für die Temperaturdokumentation und ein akustisches sowie optisches Alarmsignal. Der Innenraum besteht aus schlagfestem Kunststoff und bietet 95 Liter Nutzvolumen. Das Gerät ist mit zwei kunststoffbeschichteten Rosten ausgestattet und hat eine digitale Temperaturanzeige. Der Kirsch FROSTER LABEX-96 PRO-ACTIVE ist ideal für den Einsatz in Laboren, Kliniken und im Gesundheitswesen.', 'category': 'Labortiefkühlschrank', 'manufac

100%|██████████| 151/151 [1:38:00<00:00, 38.94s/it]

{'summary': 'Der Liebherr SRPvg 1402 Performance ist ein leistungsstarker Laborkühlschrank mit einem Bruttovolumen von 1.391 Litern und einem einstellbaren Temperaturbereich von -2°C bis +16°C. Das Gerät verfügt über eine dynamische Umluftkühlung, die eine gleichmäßige Temperaturverteilung im gesamten Innenraum gewährleistet, mit einer maximalen Temperaturabweichung von ±3°C. Der SRPvg 1402 ist mit einem Alarm-Relais und einer nachrüstbaren WiFi-/LAN-Schnittstelle ausgestattet, die die Integration in das Liebherr SmartMonitoring-System ermöglicht. Mit einem jährlichen Energieverbrauch von 464 kWh und einer Nutzungsdauer von bis zu 15 Jahren bietet der SRPvg 1402 Performance eine nachhaltige und zuverlässige Lösung für anspruchsvolle Anwendungen in medizinischen Einrichtungen, Forschungslaboren und Apotheken.', 'category': 'Laborkühlschrank', 'manufacturer': 'Liebherr', 'title': 'SRPvg 1402 Performance'}
['Der Liebherr SRPvg 1402 Performance ist ein Laborkühlschrank, der speziell für me

## Quality Checks

Wir müssen zuletzt noch in die Chunks schauen ob sie die wichtigsten Rahmenbedinungen einhalten. Vor allem die Länge da die meisten Models eine beschränkte Anzahl an Tokes verarbeiten kann und die darf von den Chunks nicht überschritten werden.

In [4]:
def stats(df, treshhold, name):

    df['length'] = df['document'].str.len()
    small = df[df['length'] < treshhold]

    print(f'=== {name} ===')
    print(f"{df['length'].describe()}")
    print(f'TOTAL CHUNKS: {len(df)}')
    print(f'SHORT CHUNKS: {len(small)}')

    for idx, row in small.iterrows():
        
        print(f"{row['metadata']['product_title']}")
        print(f"Document: {row['document']}")
        print(f"Length: {row['length']}")
        print(f'--- --- ---')

    print('\n')


In [5]:
stats(summs_df, 200, 'Summs')
stats(descs_df, 200, 'Descriptions')
stats(specs_df, 100, 'Specs')

=== Summs ===
count     151.000000
mean      903.476821
std       169.975756
min       556.000000
25%       786.000000
50%       892.000000
75%      1023.000000
max      1360.000000
Name: length, dtype: float64
TOTAL CHUNKS: 151
SHORT CHUNKS: 0


=== Descriptions ===
count    903.000000
mean     353.521595
std      110.237676
min       85.000000
25%      282.000000
50%      342.000000
75%      411.500000
max      926.000000
Name: length, dtype: float64
TOTAL CHUNKS: 903
SHORT CHUNKS: 43
SFFsg 5501 Performance
Document: Die Geräte sind super-effizient im Energieverbrauch und erfüllen die Sicherheitsstandards nach IEC 61010-2-011 sowie die ATEX-Richtlinie (SRFfg/SFFfg).
Length: 151
--- --- ---
BL-720 ULTIMATE
Document: Mit einem Kühlinhalt von 700 Litern und einem Normalverbrauch von 1,56 kWh in 24 Stunden ist der BL-720 ein effizientes und zuverlässiges Gerät für die Blutkonservierung.
Length: 170
--- --- ---
Tiefkühltruhe TT 50-90
Document: Die FRYKA Tiefkühltruhe TT 50-90 ist ideal fü

An dieser Stelle wurden weitere Verbesserungen an den Promts vorgenommen und getestet... Mit den Specs muss noch eine andere Strategie ausgedacht werden. Jetzt erstmal mit den kleinen Chunks probieren. 

## Speichern

In [6]:
summs_df.to_json('summs_chunks.jsonl', orient='records', lines=True, force_ascii=False)
descs_df.to_json('descs_chunks.jsonl', orient='records', lines=True, force_ascii=False)
specs_df.to_json('specs_chunks.jsonl', orient='records', lines=True, force_ascii=False)